<a href="https://colab.research.google.com/github/Michelleagl/Series_de_Tiempo/blob/main/ETS_IPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf


In [16]:
!pip install statsforecast


In [17]:

ticker = yf.Tickers(["LABB.MX", "VESTA.MX", "ORBIA.MX", "GFNORTEO.MX", "BBAJIOO.MX", "MEGACPO.MX",
                     "AC.MX", "GCC.MX", "GENTERA.MX", "CUERVO.MX", "AMXB.MX", "GCARSOA1.MX",
                     "KIMBERA.MX", "GMEXICOB.MX", "CHDRAUIB.MX", "GRUMAB.MX", "KOFUBL.MX",
                     "RA.MX", "BOLSAA.MX", "ALSEA.MX", "TLEVISACPO.MX", "Q.MX", "BIMBOA.MX",
                     "PINFRA.MX", "GAPB.MX", "ASURB.MX", "OMAB.MX", "CEMEXCPO.MX"])
data = ticker.history(start= '2015-01-01', end= '2023-12-31', interval='1mo')['Close']
# Fecha en meses
data.head()

[*********************100%***********************]  28 of 28 completed


Ticker,AC.MX,ALSEA.MX,AMXB.MX,ASURB.MX,BBAJIOO.MX,BIMBOA.MX,BOLSAA.MX,CEMEXCPO.MX,CHDRAUIB.MX,CUERVO.MX,...,KOFUBL.MX,LABB.MX,MEGACPO.MX,OMAB.MX,ORBIA.MX,PINFRA.MX,Q.MX,RA.MX,TLEVISACPO.MX,VESTA.MX
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00+00:00,65.702621,37.795326,11.671397,152.225357,NaN,34.200211,15.430018,11.529756,37.656048,NaN,...,NaN,19.345701,38.958519,38.346153,33.406322,130.751801,NaN,56.302013,88.730598,23.547871
2015-02-01 00:00:00+00:00,70.285500,43.193314,11.671397,159.254852,NaN,37.858753,16.780148,13.123650,44.382278,NaN,...,NaN,13.299150,39.022575,39.573227,35.030018,138.726151,NaN,57.645134,92.414070,23.328478
2015-03-01 00:00:00+00:00,69.477180,42.006325,11.124301,159.635452,NaN,38.791317,16.133081,12.534602,41.835014,NaN,...,NaN,11.775289,42.517673,41.436157,32.118538,126.094269,NaN,56.418171,91.677368,23.035954
2015-04-01 00:00:00+00:00,69.892456,43.400566,11.737048,168.668915,NaN,36.944118,18.360479,13.333282,41.735298,NaN,...,NaN,14.749671,45.450417,42.735737,35.423912,136.695770,NaN,63.874294,101.554512,23.328478
2015-05-01 00:00:00+00:00,70.519691,43.626667,11.744343,176.417770,NaN,37.374535,16.991688,12.990942,42.756989,NaN,...,NaN,13.535472,46.048359,40.197929,37.616749,130.000061,NaN,61.688995,105.901924,22.910902


In [18]:
# Re programar el formato de la fecha
data.index = data.index.strftime('%Y-%m')
data.index = pd.to_datetime(data.index)


In [19]:
# De columnas las convierte en filas des-pivot (todo en forma vertical), para poder usar más adelante la paqueteria
data = data.melt(ignore_index=False)



In [20]:
data.head()


,Ticker,value
Date,,
2015-01-01,AC.MX,65.702621
2015-02-01,AC.MX,70.285500
2015-03-01,AC.MX,69.477180
2015-04-01,AC.MX,69.892456
2015-05-01,AC.MX,70.519691


In [21]:
df = pd.DataFrame({"ds": data.index,
                   "y": data['value'],
                   "unique_id": data['Ticker']})
df.reset_index(drop=True, inplace=True) # Para quitar el indice de lo que acabamos de crear



In [22]:
df.head() # Renombrando las columnas


,ds,y,unique_id
0,2015-01-01,65.702621,AC.MX
1,2015-02-01,70.285500,AC.MX
2,2015-03-01,69.477180,AC.MX
3,2015-04-01,69.892456,AC.MX
4,2015-05-01,70.519691,AC.MX


In [23]:
# Limpiar valores (NAs - sin valores, limpiar data para poder pronosticar)
df.dropna(inplace=True)


In [24]:
train = df[df.ds<'2023-01-01']
test = df[df.ds>='2023-01-01']

In [25]:
df.head()

,ds,y,unique_id
0,2015-01-01,65.702621,AC.MX
1,2015-02-01,70.285500,AC.MX
2,2015-03-01,69.477180,AC.MX
3,2015-04-01,69.892456,AC.MX
4,2015-05-01,70.519691,AC.MX


In [26]:
train.shape, test.shape


((2573, 3), (336, 3))

In [33]:
from statsforecast.models import AutoETS
from statsforecast.core import StatsForecast

In [34]:
season_length = 12 # Datos mensuales
horizon = len(test['ds'].unique()) # Datos a predecir

models = [AutoETS(season_length=season_length, model='ZZZ')] # ZZZ es no se nada para que la misma libreria


In [35]:
sf = StatsForecast(df=train,
                   models=models,
                   freq='MS',
                   n_jobs=-1)

/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:476: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(


In [36]:
sf.fit()

/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:476: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(


StatsForecast(models=[AutoETS])

In [39]:
sf = StatsForecast(df=train,
                   models=models,
                   freq='MS',
                   n_jobs=-1)
sf.fit() # Fit the model within this cell itself

sf.fitted_values[0,0].model_.keys() # Access the fitted values using fitted_values

# encontrar cuales son los modelos que se ajustan peor

/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:476: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/statsforecast/core.py:476: FutureWarning: The `df` argument of the StatsForecast constructor as well as reusing stored dfs from other methods is deprecated and will raise an error in a future version. Please provide the `df` argument to the corresponding method instead, e.g. fit/forecast.
  warnings.warn(


AttributeError: 'StatsForecast' object has no attribute 'fitted_values'